Reservation first

In [1]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import hardware

context.version = "1.0" # required during transition
context.choose_site(default="CHI@UC")
context.choose_project()
username = os.getenv('USER') # all exp resources will have this prefix

In [2]:
node_type = "gpu_rtx_6000"

In [3]:
gpu_start_times = [n.next_free_timeslot()[0] for n in hardware.get_nodes(node_type=node_type)]
current_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(minutes=1)
if min(gpu_start_times) > current_time:
    lease_start = min(gpu_start_times)
    print(f"There is no {node_type} available now, you can request one starting at {str(lease_start)} (UTC).")
else:
    lease_start = current_time
    print(f"A {node_type} IS available now, your lease will start right away.")


A gpu_rtx_6000 IS available now, your lease will start right away.


In [4]:
l = lease.Lease(f"repro-{username}-{node_type}", duration=datetime.timedelta(hours=4),
                start_date = lease_start  )
l.add_node_reservation(node_type = node_type, amount = 1) 
l.add_fip_reservation(1) 
l.submit(idempotent=True)

Waiting for lease to start...


Lease repro-ezrajoanerinata_gmail_com-gpu_rtx_6000 has reached status active


In [6]:
# continue here, whether using a lease created just now or one created earlier
l = lease.get_lease(f"repro-{username}-{node_type}")

CHIValueError: Lease not found maching repro-ezrajoanerinata_gmail_com-gpu_rtx_6000

In [ ]:
s = server.Server(
    f"repro-{username}-{node_type}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

In [ ]:
reserved_fip = l.get_reserved_floating_ips()[0]
s.associate_floating_ip(reserved_fip)

In [ ]:
s.check_connectivity(host=reserved_fip)

In [ ]:
print("ssh cc@" + reserved_fip)

Put the output above into this terminal (File -> New -> Terminal)

In [10]:
from chi import ssh
node = ssh.Remote(reserved_fip) 

In [11]:
node.run("curl -sSL https://get.docker.com/ | sudo sh")
node.run("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.212: b'f48a371c3ee6a0415f8db5d08fd8375a'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

In [12]:
node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new group membership
node.run("docker run hello-world")

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
e6590344b1a5: Pulling fs layer
e6590344b1a5: Verifying Checksum
e6590344b1a5: Download complete
e6590344b1a5: Pull complete
Digest: sha256:dd01f97f252193ae3210da231b1dca0cffab4aadb3566692d6730bf93f123a48
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

In [13]:
# get NVIDIA container toolkit 
node.run("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
node.run("sudo apt update")
node.run("sudo apt-get install -y nvidia-container-toolkit")
node.run("sudo nvidia-ctk runtime configure --runtime=docker")
node.run("sudo systemctl restart docker")

deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/stable/deb/$(ARCH) /
#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/experimental/deb/$(ARCH) /


Hit:1 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:2 https://download.docker.com/linux/ubuntu noble InRelease
Get:3 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:6 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [19.5 kB]
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 277 kB in 1s (224 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
14 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libnvidia-container-tools libnvidia-container1 nvidia-c

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5933 kB in 2s (3886 kB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 113813 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.7-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.7-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.7-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.7-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.7-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.7-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.7-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.7-1) ...
Setting up nvidia-container-toolkit-base (1.17.7-1) ...
Setting up libnvidia-container1:amd64 (1.17.7-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-05-22T04:56:35Z" level=info msg="Config file does not exist; using empty config"
time="2025-05-22T04:56:35Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-05-22T04:56:35Z" level=info msg="It is recommended that docker daemon be restarted."


<Result cmd='sudo systemctl restart docker' exited=0>

In [14]:
# check that we can see GPU from inside container
node.run("docker run --rm --gpus all ubuntu nvidia-smi")

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu
0622fac788ed: Pulling fs layer
0622fac788ed: Verifying Checksum
0622fac788ed: Download complete
0622fac788ed: Pull complete
Digest: sha256:6015f66923d7afbc53558d7ccffd325d43b4e249f41a6e93eef074c9505d2233
Status: Downloaded newer image for ubuntu:latest


Thu May 22 04:56:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 6000                Off |   00000000:3B:00.0 Off |                  Off |
| 33%   23C    P8              4W /  260W |       2MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

<Result cmd='docker run --rm --gpus all ubuntu nvidia-smi' exited=0>

Installing PyTorch

In [15]:
node.run("docker run -d -p 8888:8888 --rm --gpus all quay.io/jupyter/pytorch-notebook:cuda12-latest")

Unable to find image 'quay.io/jupyter/pytorch-notebook:cuda12-latest' locally
cuda12-latest: Pulling from jupyter/pytorch-notebook
f03f49e66a78: Pulling fs layer
c9f1dea21db4: Pulling fs layer
30b4d653e3ef: Pulling fs layer
4f4fb700ef54: Pulling fs layer
85bab3a7dedd: Pulling fs layer
028098059d19: Pulling fs layer
925efed95603: Pulling fs layer
70c9261e1635: Pulling fs layer
db4b07d34255: Pulling fs layer
9f853acf98a4: Pulling fs layer
2500b0bb8b45: Pulling fs layer
32627e9e77a2: Pulling fs layer
3010714c694d: Pulling fs layer
3cf9b47fb7b9: Pulling fs layer
3f39a091ed34: Pulling fs layer
01dea47f2f1f: Pulling fs layer
e7f51807ced7: Pulling fs layer
44bf8bc6d931: Pulling fs layer
85bab3a7dedd: Waiting
db4b07d34255: Waiting
4f4fb700ef54: Waiting
32627e9e77a2: Waiting
2500b0bb8b45: Waiting
cad571f06a45: Pulling fs layer
028098059d19: Waiting
3010714c694d: Waiting
05c9dbba26c4: Pulling fs layer
3f39a091ed34: Waiting
925efed95603: Waiting
cad571f06a45: Waiting
9f853acf98a4: Waiting
5e185a4

70d025eaa9d384118a1496cee5ca8631e18d656da8aa61c26e41bdec364225a9


<Result cmd='docker run -d -p 8888:8888 --rm --gpus all quay.io/jupyter/pytorch-notebook:cuda12-latest' exited=0>

In [16]:
node.run("docker ps -q | xargs -L 1 docker logs")

Entered start.sh with args: start-notebook.py
Running hooks in: /usr/local/bin/start-notebook.d as uid: 1000 gid: 100
Done running hooks in: /usr/local/bin/start-notebook.d
Running hooks in: /usr/local/bin/before-notebook.d as uid: 1000 gid: 100
Sourcing shell script: /usr/local/bin/before-notebook.d/10activate-conda-env.sh
Done running hooks in: /usr/local/bin/before-notebook.d
Executing the command: start-notebook.py
[I 2025-05-22 05:06:40.003 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2025-05-22 05:06:40.006 ServerApp] jupyter_server_mathjax | extension was successfully linked.
[I 2025-05-22 05:06:40.008 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2025-05-22 05:06:40.012 ServerApp] jupyterlab | extension was successfully linked.


Executing: jupyter lab


[I 2025-05-22 05:06:40.012 ServerApp] jupyterlab_git | extension was successfully linked.
[I 2025-05-22 05:06:40.014 ServerApp] nbclassic | extension was successfully linked.
[I 2025-05-22 05:06:40.014 ServerApp] nbdime | extension was successfully linked.
[I 2025-05-22 05:06:40.017 ServerApp] notebook | extension was successfully linked.
[I 2025-05-22 05:06:40.020 ServerApp] Writing Jupyter server cookie secret to /home/jovyan/.local/share/jupyter/runtime/jupyter_cookie_secret
[I 2025-05-22 05:06:40.197 ServerApp] notebook_shim | extension was successfully linked.
[W 2025-05-22 05:06:40.212 ServerApp] WARNING: The Jupyter server is listening on all IP addresses and not using encryption. This is not recommended.
[I 2025-05-22 05:06:40.213 ServerApp] notebook_shim | extension was successfully loaded.
[I 2025-05-22 05:06:40.214 ServerApp] jupyter_lsp | extension was successfully loaded.
[I 2025-05-22 05:06:40.215 ServerApp] jupyter_server_mathjax | extension was successfully loaded.
[I 2

<Result cmd='docker ps -q | xargs -L 1 docker logs' exited=0>

Search the output like : http://127.0.0.1:8888/lab?token=XXXXXXXXXXXXXXXXXXXXX. but change the 127.0.0.1 with your floating ip, and go into that url. We will do the reproduce on that url.

Do this after done with all the work (reproducing the paper)

In [19]:
#stop the container
node.run("docker ps -q | xargs -L 1 docker container stop")

EOFError: 

In [20]:
# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, magic, context

context.version = "1.0" # required during transition

context.choose_site(default="CHI@UC")
context.choose_project()
username = os.getenv('USER') # all exp resources will have this prefix

node_type = "gpu_rtx_6000"
l = lease.get_lease(f"repro-{username}-{node_type}")

In [ ]:
# un-comment to free resources
chi.magic.cleanup_resources(l.id)
